In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import mglearn

from IPython.display import display

%matplotlib inline

# Копирам си модела от лекцията

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelBinarizer

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[[self.key]]
    
class LabelBinarizerPipelineFriendly(LabelBinarizer):
    def fit(self, X, y=None):
        super().fit(X)

    def transform(self, X, y=None):
        return super().transform(X)

    def fit_transform(self, X, y=None):
        return super().fit(X).transform(X)
    
class StringImputer(TransformerMixin):
    def fit(self, X, *_):
        self.modes = X.mode().iloc[0]
        return self
        
    def transform(self, X, y=None):
        return X.fillna(self.modes)

In [10]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [11]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

model = Pipeline([
    ('union', FeatureUnion([
        ('age', Pipeline([
            ('select', ItemSelector('Age')),
            ('imputer', Imputer(strategy='mean')),
            ('scaler', StandardScaler()),
        ])),
        ('gender', Pipeline([
            ('select', ItemSelector('Sex')),
            ('imputer', StringImputer()),
            ('encoder', LabelBinarizerPipelineFriendly()),
        ])),
        ('embarked', Pipeline([
            ('select', ItemSelector('Embarked')),
            ('imputer', StringImputer()),
            ('encoder', LabelBinarizerPipelineFriendly()),
        ])),
        ('sibsp', Pipeline([
            ('select', ItemSelector('SibSp')),
            ('scaler', StandardScaler()),
        ])),
        ('parch', Pipeline([
            ('select', ItemSelector('Parch')),
            ('scaler', StandardScaler()),
        ])),
    ])),
    ('classifier', SVC())
])

scores = cross_val_score(model, train, train['Survived'])
print(scores)
print(scores.mean())

[0.81144781 0.83164983 0.82491582]
0.8226711560044894


# Сега ще се опитам да го подобря..
## Ще пусна Grid Search върху 3 алгоритъма (SVC, LogisticRegression, RandomForest)

Използвам същата трансформация на данните

In [12]:
feature_union = FeatureUnion([
    ('age', Pipeline([
        ('select', ItemSelector('Age')),
        ('imputer', Imputer(strategy='mean')),
        ('scaler', StandardScaler()),
    ])),
    ('gender', Pipeline([
        ('select', ItemSelector('Sex')),
        ('imputer', StringImputer()),
        ('encoder', LabelBinarizerPipelineFriendly()),
    ])),
    ('embarked', Pipeline([
        ('select', ItemSelector('Embarked')),
        ('imputer', StringImputer()),
        ('encoder', LabelBinarizerPipelineFriendly()),
    ])),
    ('sibsp', Pipeline([
        ('select', ItemSelector('SibSp')),
        ('scaler', StandardScaler()),
    ])),
    ('parch', Pipeline([
        ('select', ItemSelector('Parch')),
        ('scaler', StandardScaler()),
    ])),
])

Разделям на трейн и тест и ги трансформирам

In [13]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(train, train["Survived"], random_state=0)

transformer = feature_union.fit(X_train)
X_train_transformed = transformer.transform(X_train)
X_test_transformed = transformer.transform(X_test)

Пускам Грид Сърч над 3те алгоритъма.. Нека най-добрият победи.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('classifier', SVC())
])

grid = [
    {
        'classifier': [SVC()],
        'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
    },
    {
        'classifier': [RandomForestClassifier()],
        'classifier__max_features': [1, 2, 3, 5],
        'classifier__n_estimators': [10, 50, 100, 200]
    },
    {
        'classifier': [LogisticRegression()],
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
    }
]

In [8]:
from sklearn.model_selection import GridSearchCV

search = GridSearchCV(pipeline, grid, cv=5)
search.fit(X_train_transformed, y_train)

print("Best params:\n{}\n".format(search.best_params_))
print("Best cross-validation score: {:.2f}".format(search.best_score_))

Best params:
{'classifier': SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), 'classifier__C': 1, 'classifier__gamma': 1}

Best cross-validation score: 0.82


# Изглежда, че модела от лекцията е най-добрия от всички

# Ще се наложи да поиграем с фийчърите

## Заблязах, че поради някаква причина Fare липсва. Нека го добавим

In [14]:
feature_union = FeatureUnion([
    ('age', Pipeline([
        ('select', ItemSelector('Age')),
        ('imputer', Imputer(strategy='mean')),
        ('scaler', StandardScaler()),
    ])),
    ('gender', Pipeline([
        ('select', ItemSelector('Sex')),
        ('imputer', StringImputer()),
        ('encoder', LabelBinarizerPipelineFriendly()),
    ])),
    ('embarked', Pipeline([
        ('select', ItemSelector('Embarked')),
        ('imputer', StringImputer()),
        ('encoder', LabelBinarizerPipelineFriendly()),
    ])),
    ('sibsp', Pipeline([
        ('select', ItemSelector('SibSp')),
        ('scaler', StandardScaler()),
    ])),
    ('parch', Pipeline([
        ('select', ItemSelector('Parch')),
        ('scaler', StandardScaler()),
    ])),
    ('fare', Pipeline([
        ('select', ItemSelector('Fare')),
        ('scaler', StandardScaler())
    ]))
])

## Да претренираме..

In [13]:
transformer = feature_union.fit(X_train)
X_train_transformed = transformer.transform(X_train)
X_test_transformed = transformer.transform(X_test)
print(X_train_transformed.shape)

(668, 8)


Може би трябва да извадя това във функция

In [11]:
search.fit(X_train_transformed, y_train)

print("Best params:\n{}\n".format(search.best_params_))
print("Best cross-validation score: {:.2f}".format(search.best_score_))

Best params:
{'classifier': SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), 'classifier__C': 1, 'classifier__gamma': 0.1}

Best cross-validation score: 0.82


# Няма промяна....

# Опит с невронна мрежа

In [3]:
from keras.layers import Dense, Dropout, GlobalMaxPooling1D
from keras.models import Sequential
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [18]:
X = train.drop('Survived', axis=1)
X[:10]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [22]:
Y = train['Survived']
Y.shape

(891,)

In [21]:
x_transformed = feature_union.fit_transform(X)
x_transformed.shape

(891, 8)

In [39]:
NN_model = Sequential([
    Dense(8,input_shape=(8,)),
    Dense(8, activation="relu"),Dense(5, activation="relu"),
    Dense(1, activation="softmax")
])

NN_model.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_27 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_28 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 6         
Total params: 195
Trainable params: 195
Non-trainable params: 0
_________________________________________________________________


In [40]:
from keras.callbacks import EarlyStopping

stopping = EarlyStopping(patience=2)

In [42]:
NN_model.fit(x_transformed, Y, validation_split=0.2, epochs=200)

Train on 712 samples, validate on 179 samples
Epoch 1/200
712/712 [==============================] - 0s 88us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 2/200
712/712 [==============================] - 0s 67us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 3/200
712/712 [==============================] - 0s 57us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 4/200
712/712 [==============================] - 0s 66us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 5/200
712/712 [==============================] - 0s 67us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 6/200
712/712 [==============================] - 0s 61us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 7/200
712/712 [==============================] - 0s 67us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 8/2

Epoch 61/200
712/712 [==============================] - 0s 60us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 62/200
712/712 [==============================] - 0s 59us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 63/200
712/712 [==============================] - 0s 63us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 64/200
712/712 [==============================] - 0s 82us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 65/200
712/712 [==============================] - 0s 79us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 66/200
712/712 [==============================] - 0s 79us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 67/200
712/712 [==============================] - 0s 72us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 68/200
712/712 [==========================

712/712 [==============================] - 0s 70us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 122/200
712/712 [==============================] - 0s 66us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 123/200
712/712 [==============================] - 0s 65us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 124/200
712/712 [==============================] - 0s 49us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 125/200
712/712 [==============================] - 0s 58us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 126/200
712/712 [==============================] - 0s 57us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 127/200
712/712 [==============================] - 0s 52us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 128/200
712/712 [==============================] 

Epoch 181/200
712/712 [==============================] - 0s 61us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 182/200
712/712 [==============================] - 0s 71us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 183/200
712/712 [==============================] - 0s 57us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 184/200
712/712 [==============================] - 0s 65us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 185/200
712/712 [==============================] - 0s 87us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 186/200
712/712 [==============================] - 0s 92us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 187/200
712/712 [==============================] - 0s 69us/step - loss: 9.7177 - acc: 0.3904 - val_loss: 10.2423 - val_acc: 0.3575
Epoch 188/200
712/712 [==================

# ?? To be continued.. 